In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pickle
import pandas as pd
import glob
import numpy as np
from deepdiff import DeepDiff

In [4]:
import sys

In [5]:
sys.path.append('/home/akumar/nse/neural_control')
from loaders import load_sabes

### Past decoding

In [5]:
peanut_decoding_files = glob.glob('/mnt/Secondary/data/peanut_decoding01/peanut_decoding01_*.dat')

In [8]:
result_list = []
for file in peanut_decoding_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)

In [9]:
peanut_decoding_df = pd.DataFrame(result_list)

In [12]:
with open('/home/akumar/nse/neural_control/data/peanut_decoding_df.dat', 'wb') as f:
    f.write(pickle.dumps(peanut_decoding_df))

In [13]:
sabes_decoding_files = glob.glob('/mnt/Secondary/data/sabes_decoding01/sabes_decoding01_*.dat')

In [18]:
result_list = []
for file in sabes_decoding_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)

In [19]:
sabes_decoding_df = pd.DataFrame(result_list)

In [21]:
with open('/home/akumar/nse/neural_control/data/sabes_decoding_df.dat', 'wb') as f:
    f.write(pickle.dumps(sabes_decoding_df))

### Conslidating 4/27

In [5]:
# indyparametric_files = glob.glob('/mnt/Secondary/data/indy_dimreduc_parametric/indy_dimreduc_parametric_*.dat')
argfiles = glob.glob('/mnt/Secondary/data/indy_dimreduc_parametric/arg*.dat')

In [8]:
result_list = []
for argfile in argfiles:

    # Open up the arg files
    with open(argfile, 'rb') as f:
        args = pickle.load(f)

    rfile = args['results_file']
    with open(rfile, 'rb') as f:
        result = pickle.load(f)

    for result_ in result:
        for k, v in args.items():
            result_[k] = v
    
    result_list.extend(result)
dimreduc_df = pd.DataFrame(result_list)

In [9]:
with open('/home/akumar/nse/neural_control/data/indy_parametric_dimreduc_df.dat', 'wb') as f:
    f.write(pickle.dumps(dimreduc_df))

In [15]:
dimreduc_files = glob.glob('/mnt/Secondary/data/indy_trialized_dimreduc/indy_trialized_dimreduc_*.dat')

In [17]:
result_list = []
for file in dimreduc_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)
dimreduc_df = pd.DataFrame(result_list)

In [18]:
with open('/home/akumar/nse/neural_control/data/indy_trialized_dimreduc_df.dat', 'wb') as f:
    f.write(pickle.dumps(dimreduc_df))

In [3]:
decoding_files = glob.glob('/mnt/Secondary/data/peanut_decoding_fcca/peanut_decoding_fcca_*.dat')

In [5]:
result_list = []
for file in decoding_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)
decoding_df = pd.DataFrame(result_list)

In [6]:
with open('/home/akumar/nse/neural_control/data/peanut_fcca_decoding_df.dat', 'wb') as f:
    f.write(pickle.dumps(decoding_df))

In [3]:
decoding_files = glob.glob('/mnt/Secondary/data/indydimreducparam_decoding/indydimreducparam_decoding_*.dat')

In [4]:
result_list = []


result_list = []
for file in decoding_files:
    with open(file, 'rb') as f:
        result = pickle.load(f)
    result_list.extend(result)
decoding_df = pd.DataFrame(result_list)

In [5]:
import hickle

In [3]:
argfiles = glob.glob('/mnt/Secondary/data/indydimreducparam_decoding/arg*.dat')

In [4]:
loader_args = []
for argfile in argfiles:
    with open(argfile, 'rb') as f:
        args = pickle.load(f)
    
    # Track down the arg file associated with the dimreduc result
    dimreducpath = '/'.join(args['task_args']['dimreduc_file'].split('/')[:-1])
    dimreducno = args['task_args']['dimreduc_file'].split('.dat')[0].split('_')[-1]
    argfile = '%s/arg%s.dat' % (dimreducpath, dimreducno)    
    
    with open(argfile, 'rb') as f:
        dimreduc_args = pickle.load(f)
    
    loader_args.append(dimreduc_args['loader_args'])

In [5]:
unique_loader_args = []
for loader_arg in loader_args:
    if not unique_loader_args:
        unique_loader_args.append(loader_arg)
    else:
        diffs = []
        for ula in unique_loader_args:
            diffs.append(DeepDiff(ula, loader_arg))
        
        diffs = np.array([len(d) for d in diffs])
        if np.all(diffs > 0):
            unique_loader_args.append(loader_arg)
        

In [6]:
len(unique_loader_args)

16

In [7]:
import pdb

In [10]:
# Separately agregarate and save for each loader arg
for i, ula in enumerate(unique_loader_args):
    results = []
    for loader_arg, argfile in zip(loader_args, argfiles):
        if len(DeepDiff(loader_arg, ula)) == 0:
            # Open up the results file
            with open(argfile, 'rb') as f:
                args = pickle.load(f)

            # Open up the dimreduc args save the args with the results
            # Track down the arg file associated with the dimreduc result
            dimreducpath = '/'.join(args['task_args']['dimreduc_file'].split('/')[:-1])
            dimreducno = args['task_args']['dimreduc_file'].split('.dat')[0].split('_')[-1]
            argfile = '%s/arg%s.dat' % (dimreducpath, dimreducno)    
            
            with open(argfile, 'rb') as f:
                dimreduc_args = pickle.load(f)



            results_file = args['results_file']
            with open(results_file, 'rb') as f:
                result = pickle.load(f)
                for result_ in result:
                    for k, v in args.items():
                        result_[k] = v
                    for k, v in dimreduc_args.items():
                        result_[k] = v

            results.extend(result)

    df = pd.DataFrame(results)
    with open('/home/akumar/nse/neural_control/data/indy_parametric_decoding/df%d.dat' % i, 'wb') as f:
        f.write(pickle.dumps(df))

In [12]:
df.iloc[0]

dim                                                               10
fold_idx                                                           0
train_idxs         [11706, 11707, 11708, 11709, 11710, 11711, 117...
test_idxs          [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
dimreduc_method                                                  DCA
dimreduc_args                                  {'T': 3, 'n_init': 5}
coef               [[0.043348193288712804, 0.06172647904631565, -...
score                                                      68.058941
decoder                                                           lr
decoder_args       {'trainlag': 4, 'testlag': 4, 'decoding_window...
decoder_obj                         LinearRegression(normalize=True)
r2                 [0.14425542097701782, 0.2055141988659695, 0.02...
data_file             /mnt/Secondary/data/sabes/indy_20160630_01.mat
loader                                                         sabes
loader_args        {'bin_width': 2

In [7]:
argfiles = glob.glob('/mnt/Secondary/data/indytrialized_decoding/arg*.dat')

In [10]:
result_list = []
not_done = []
for argfile in argfiles:
    with open(argfile, 'rb') as f:
        args = pickle.load(f)

    # Open up the dimreduc args save the args with the results
    # Track down the arg file associated with the dimreduc result
    dimreducpath = '/'.join(args['task_args']['dimreduc_file'].split('/')[:-1])
    dimreducno = args['task_args']['dimreduc_file'].split('.dat')[0].split('_')[-1]
    argfile = '%s/arg%s.dat' % (dimreducpath, dimreducno)    
    
    with open(argfile, 'rb') as f:
        dimreduc_args = pickle.load(f)

    results_file = args['results_file']
    try:
        with open(results_file, 'rb') as f:
            result = pickle.load(f)
            for result_ in result:
                for k, v in args.items():
                    result_[k] = v
                for k, v in dimreduc_args.items():
                    result_[k] = v
    except FileNotFoundError:
        not_done.append(results_file)
    result_list.extend(result)
decoding_df = pd.DataFrame(result_list)

In [13]:
idxs = [int(n.split('.dat')[0].split('_')[-1]) for n in not_done]

In [16]:
# Files that did not complete here are results of T being too long for trialized trajectories

In [15]:
np.sort(idxs)

array([   1,  189,  193,  197,  205,  209,  213,  217,  221,  229,  233,
        237,  433,  437,  441,  449,  453,  457,  461,  465,  469,  637,
        909, 1245, 1249, 1253, 1397, 1441, 1445, 1449, 1457, 1461, 1465,
       1469, 1473, 1477])

In [17]:
with open('/home/akumar/nse/neural_control/data/indytrialized_decoding_df.dat', 'wb') as f:
    f.write(pickle.dumps(decoding_df))

### Filter loco UoI data files and save them

In [3]:
data_path = '/mnt/Secondary/data/sabes'    
 
# These are the data files that contain both M1 and S1 recordings.
data_files = glob.glob('%s/loco*' % data_path)
data_files = [data_files[0], data_files[5]]
loader_args = {'bin_width':50, 'filter_fn':'none', 'filter_kwargs':{}, 'boxcox':0.5, 'spike_threshold':100, 'region':'S1'}


In [6]:
d1 = load_sabes(data_files[0], **loader_args)

Processing spikes


100%|██████████| 1/1 [00:14<00:00, 14.20s/it]


In [7]:
d2 = load_sabes(data_files[1], **loader_args)

Processing spikes


100%|██████████| 1/1 [00:14<00:00, 14.91s/it]


In [16]:
data_files[0].split('/')[-1]

'loco_20170210_03.mat'

In [19]:
with open('/mnt/Secondary/data/sabes/preprocessed/%s' % data_files[0].split('/')[-1], 'wb') as f:
    f.write(pickle.dumps(d1))
    f.write(pickle.dumps((data_files[0], loader_args)))

In [20]:
with open('/mnt/Secondary/data/sabes/preprocessed/%s' % data_files[1].split('/')[-1], 'wb') as f:
    f.write(pickle.dumps(d2))
    f.write(pickle.dumps((data_files[1], loader_args)))

### Consolidating CV Dimreudc with longer T

In [24]:
argfiles = glob.glob('/mnt/Secondary/data/cv_dimreduc2/arg*.dat')

In [25]:
result_list = []
for argfile in argfiles:

    # Open up the arg files
    with open(argfile, 'rb') as f:
        args = pickle.load(f)

    rfile = args['results_file']
    with open(rfile, 'rb') as f:
        result = pickle.load(f)

    for result_ in result:
        for k, v in args.items():
            result_[k] = v
    
    result_list.extend(result)
dimreduc_df = pd.DataFrame(result_list)

In [26]:
with open('/home/akumar/nse/neural_control/data/cv_dimreduc_df2.dat', 'wb') as f:
    f.write(pickle.dumps(result_list))

In [28]:
dimreduc_df.iloc[0]['dimreduc_args']

{'T': 10, 'n_init': 5}

### CV_dimreduc aross both subjects

In [5]:
argfiles = glob.glob('/mnt/Secondary/data/cv_dimreduc_both/arg*')

In [7]:
rl = []
for f_ in argfiles:
    with open(f_, 'rb') as f:
        args = pickle.load(f)
    rfile = args['results_file']
    with open(rfile, 'rb') as f:
        result = pickle.load(f)

    for result_ in result:
        for k, v in args.items():
            result_[k] = v
    
    rl.extend(result)

In [9]:
with open('/home/akumar/nse/neural_control/data/cv_dimreduc_both.dat', 'wb') as f:
    f.write(pickle.dumps(rl))

### Indy signfix

In [71]:
argfiles = glob.glob('/mnt/Secondary/data/sabes_signfix_dw5/arg*.dat')

In [72]:
len(argfiles)

56

In [73]:
import pdb

In [74]:
result_list = []
for argfile in argfiles:

    # Open up the arg files
    with open(argfile, 'rb') as f:
        args = pickle.load(f)

    rfile = args['results_file']
    with open(rfile, 'rb') as f:
        result = pickle.load(f)

    for result_ in result:
        for k, v in args.items():
            if type(k) == dict:
                for k_, v_ in k.items():
                    result_[k_] = v_
            else:
                result_[k] = v

    # Need to grab some information from the dimreduc arg file
    dimreduc_path = '/'.join(args['task_args']['dimreduc_file'].split('/')[:-1])
    dimreduc_no = args['task_args']['dimreduc_file'].split('_')[-1].split('.dat')[0]
    dimreduc_argfile = '%s/arg%s.dat' % (dimreduc_path, dimreduc_no)

    with open(dimreduc_argfile, 'rb') as f:
        dr_args = pickle.load(f)

    for result_ in result:
        for k, v in dr_args.items():
            if type(k) == dict:
                for k_, v_ in k.items():
                    result_[k_] = v_
            else:
                result_[k] = v

    result_list.extend(result)

In [75]:
len(result_list)

8400

In [76]:
result_list[0].keys()

dict_keys(['dim', 'fold_idx', 'train_idxs', 'test_idxs', 'dimreduc_method', 'dimreduc_args', 'coef', 'score', 'decoder', 'decoder_args', 'decoder_obj', 'r2', 'task_args', 'data_file', 'data_path', 'loader', 'loader_args', 'results_file'])

In [77]:
with open('/home/akumar/nse/neural_control/data/sabes_decoding_sf.dat', 'wb') as f:
    f.write(pickle.dumps(result_list))

In [61]:
rl = pd.DataFrame(result_list)

In [62]:
from utils import apply_df_filters

In [63]:
data_files = np.unique(rl['data_file'].values)

In [64]:
data_files

array(['/mnt/Secondary/data/sabes/indy_20160426_01.mat',
       '/mnt/Secondary/data/sabes/indy_20160622_01.mat',
       '/mnt/Secondary/data/sabes/indy_20160624_03.mat',
       '/mnt/Secondary/data/sabes/indy_20160627_01.mat',
       '/mnt/Secondary/data/sabes/indy_20160630_01.mat',
       '/mnt/Secondary/data/sabes/indy_20160915_01.mat',
       '/mnt/Secondary/data/sabes/indy_20160921_01.mat',
       '/mnt/Secondary/data/sabes/indy_20160930_02.mat',
       '/mnt/Secondary/data/sabes/indy_20160930_05.mat',
       '/mnt/Secondary/data/sabes/indy_20161005_06.mat',
       '/mnt/Secondary/data/sabes/indy_20161006_02.mat',
       '/mnt/Secondary/data/sabes/indy_20161007_02.mat',
       '/mnt/Secondary/data/sabes/indy_20161011_03.mat',
       '/mnt/Secondary/data/sabes/indy_20161013_03.mat',
       '/mnt/Secondary/data/sabes/indy_20161014_04.mat',
       '/mnt/Secondary/data/sabes/indy_20161017_02.mat',
       '/mnt/Secondary/data/sabes/indy_20161024_03.mat',
       '/mnt/Secondary/data/sab

In [48]:
rl.shape

(33600, 18)

In [65]:
df = apply_df_filters(rl, data_file=data_files[0])

In [66]:
df.shape

(1200, 18)

In [68]:
30 * 5 * 4 * 2

1200

In [34]:
20400 * 28

571200